### Analyzing Cafe Places in Riyadh - Cleaning Notebook

Imports

In [1]:
import pandas as pd
import numpy as np
import unicodedata
import re


Webscraped Data of coffee shops in riyadh, which will need alot of cleaning

In [2]:
def drop_index(df):
    return df.drop('index', axis=1)

def adding_id(df):
    df = df.copy()
    df['id'] = range(1, len(df) + 1)
    cols = ['id'] + [col for col in df.columns if col != 'id']
    return df[cols]

def replace_nulls(df):
    return df.replace(["Null", "null"], np.nan)

def rename_columns(df):
    return df.rename(columns={'lan': 'latitude', 'lon': 'longitude', 'coffeeName': 'raw_coffeeName'})

def convert_coords(df):
    df['latitude'] = pd.to_numeric(df['latitude'], errors='coerce')
    df['longitude'] = pd.to_numeric(df['longitude'], errors='coerce')
    return df


cafes = (
    pd.read_csv('data/raw_riyadh_cafes.csv')
    .pipe(drop_index)
    .pipe(adding_id)
    .pipe(replace_nulls)
    .pipe(rename_columns)
    .pipe(convert_coords)
)
cleaned_words_eng = []
cleaned_words_ar = []


print(cafes.shape)
cafes.head()

(2609, 8)


,id,raw_coffeeName,rating,rating_count,url,24_hours,longitude,latitude
0,1,Cacti Cafe,4.2,2212,https://www.google.com/maps/place/Cacti+Cafe/d...,True,46.735613,24.833986
1,2,فازا قهوة مختصة,4.3,889,https://www.google.com/maps/place/%D9%81%D8%A7...,False,46.697337,24.772458
2,3,ناريز,3.8,39,https://www.google.com/maps/place/%D9%86%D8%A7...,False,46.852109,24.807277
3,4,Fc Lounge - اف سي لاونج,3.5,539,https://www.google.com/maps/place/Fc+Lounge+-+...,True,46.768059,24.813115
4,5,PEAKS,4.6,25,https://www.google.com/maps/place/PEAKS/data=!...,False,46.634809,24.742045


#### functions

In [3]:
#Basic String Cleaning
def clean_str_ar(text):
    if not isinstance(text, str):
        return text
    
    text = text.lower().strip()

   
    
    text = text.replace('وو', 'و')
    text = text.replace('يي', 'ي')
    text = text.replace('اا', 'ا')

    # Remove Arabic diacritics (tashkeel), tatweel (ـ), 
    #invisible formatting characters, and non-breaking spaces
    text = re.sub(r'[\u0617-\u061A\u064B-\u0652\u0640\u200f\u200e\u00A0]', '', text)


    # remove more tashkeel
    search  = ["آ","إ","أ","ة","ى","  "]
    replace = ["ا","ا","ا","ه","ي"," "]

    for s, r in zip(search, replace):
        text = text.replace(s, r)
    
    # Convert dashes into spaces to avoid merging words
    text = text.replace('-', ' ')

    # Remove unwanted symbols, keeping /, %, ., and &
    text = re.sub(r'[^\w\s/٪.&]', '', text) 
    
    
    protected = [r'\bد\.|\bد\b']
    if not any(re.search(p, text.lower()) for p in protected):
        text = re.sub(r'\b(كافيه|كيف|الكيف|للقهوه|الكافيه|القهوه|قهوه|كافي|مقهي|مختصه|المختصه|كوفي|شوب|وقهوه)\b', '', text).strip()

    # Remove common road/location words
    text = re.sub(r'\b(قرطبه|طلبات السيارت)\b',
                   '', text).strip()

    
    # remove و if at the end
    text = re.sub(r'\bو\b\s*$', '', text).strip()

    # Case 1: Remove "&" if it's at the end of a multi-word string (e.g., "bread &")
    if text.strip() != '&':  # only clean if it's not JUST "&"
        text = re.sub(r'&\s*$', '', text).strip()   # remove trailing &
        text = re.sub(r'^&\s*', '', text).strip()   # remove leading &

    
    # remove '.' if at the end
    text = text.strip('.').strip()

    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    return text

def clean_str_eng(text):
    if not isinstance(text, str):
        return text
    
    text = text.lower().strip()

    # removes accents for english words
    text =''.join(
            c for c in unicodedata.normalize('NFKD', text)
            if not unicodedata.combining(c)
        )
    
    # Remove unwanted symbols, keeping /, %, ., and &
    text = re.sub(r'[^\w\s٪.&]', '', text) 


    # names we dont want to remove things like cafe. Ex: dr.cafe

    protected = [r'\bdr\b']

    if not any(re.search(p, text.lower()) for p in protected):
        text = re.sub(
            r'\b(coffee|coffees|cofee|cafe|cafee|coffe|caffe|cafa|caffee|reserve|shop|store|catering|the|station|speciality|specialty|an)\b',
            '', text).strip()
    
    # Remove common road/location words
    text = re.sub(r'\b(thumamah|kafd|dt|drive|branch|exit|mall|center|al munsiyah|thru|drivethru|drivethrough|qurtubah|al bustan|village|riyadh)\b',
                   '', text).strip()
    # Remove "&" if it's at the end/start of a multi-word string (e.g., "bread &")
    if text.strip() != '&':  # only clean if it's not JUST "&"
        text = re.sub(r'&\s*$', '', text).strip()   # remove trailing &
        text = re.sub(r'^&\s*', '', text).strip()   # remove leading &
        
    # remove extra spaces
    text = re.sub(r'\s+', ' ', text)

    # remove '.' if at the end
    text = text.strip('.').strip()

    # remove "and" if it is at the end of the word
    text = re.sub(r'\band\b\s*$', '', text).strip()

    return text


def split_by_language(text):
    if not isinstance(text, str):
        return pd.Series(['', ''], index=['english_name', 'arabic_name'])

    arabic = ' '.join(re.findall(r'[\u0600-\u06FF\s\.\-،؛؟]+', text)).strip()
    english = re.sub(r'[\u0600-\u06FF]+', '', text).strip()

    #  if English is just "&" and there's meaningful Arabic, treat as Arabic-only
    has_real_english = re.search(r'[A-Za-z0-9]', english)
    if english.strip() == '&' and not has_real_english and arabic:
        return pd.Series(['', text.strip()], index=['english_name', 'arabic_name'])

    return pd.Series([english, arabic], index=['english_name', 'arabic_name'])

def specific_eng_names():
    fix_words('barn','barns', 'eng_coffeeName')
    fix_words(r'dr\.?\s*caf(f)?e','dr.cafe','eng_coffeeName')
    fix_words(r'^java$|pnu','java','eng_coffeeName')
    fix_words(r'dunkin','dunkin donuts','eng_coffeeName')
    fix_words(r'\bdan\b','dan','eng_coffeeName')
    fix_words(r'mam|mama','mammabunz','eng_coffeeName')
    fix_words(r'tim hortons','tim hortons','eng_coffeeName')
    fix_words(r'4twin','4twins','eng_coffeeName')
    fix_words(r'\bmoon\b','moon','eng_coffeeName')
    fix_words(r'star sky|starsky','star sky','eng_coffeeName')
    fix_words('dana','dana','eng_coffeeName')
    fix_words(r'narees', 'narees', 'eng_coffeeName')
    fix_words('molten','molten chocolate','eng_coffeeName')
    fix_words(r'mercato','mercato','eng_coffeeName')
    fix_words(r'^rose$|rose lounge','rose','eng_coffeeName')
    fix_words(r'24','24cafe','eng_coffeeName')
    fix_words(r'coffeeholics','coffeeholics roastery','eng_coffeeName')
    fix_words(r'style','style','eng_coffeeName')
    fix_words(r'window','window','eng_coffeeName') 
    fix_words(r'mezaj','mezaj magribi','eng_coffeeName')
    fix_words(r'wacafe','wacafe','eng_coffeeName')
    fix_words(r'half|12 m','half million','eng_coffeeName')
    fix_words('a1coffee','a1','eng_coffeeName')
    fix_words(r'white m.*', 'white mustache','eng_coffeeName')
    fix_words(r'ramira*', 'ramira','eng_coffeeName')
    fix_words(r'caribou','caribou','eng_coffeeName')
    fix_words('gloria jeans','gloria jeans','eng_coffeeName')
    fix_words('bateel','bateel','eng_coffeeName')
    fix_words('krispy','krispy kreme','eng_coffeeName')
    fix_words('kyan','kyan','eng_coffeeName')
    fix_words('btw','btw','eng_coffeeName')
    fix_words(r'mark|m&s', 'm&s','eng_coffeeName')
    fix_words('starbuck','starbucks','eng_coffeeName')
    fix_words(r'mac|mc','mccafe','eng_coffeeName')
    
def specific_ar_names():
    fix_words(r'بارن','بارنز','ar_coffeeName')
    fix_words(r'\bد\.|\bد\b','د.كيف','ar_coffeeName')
    fix_words('عنوان','عنوان','ar_coffeeName')
    fix_words('دانه','دانه','ar_coffeeName')
    fix_words(r'\bتاج\b', 'تاج','ar_coffeeName')
    fix_words(r'ارابيكا ستار|اربيكا استار','ارابيكا ستار','ar_coffeeName')
    fix_words(r'دانكن','دانكن دونتس','ar_coffeeName')
    fix_words('عنوان','عنوان','ar_coffeeName')
    fix_words('دانه','دانه','ar_coffeeName')
    fix_words(r'\bتاج\b', 'تاج','ar_coffeeName')
    fix_words(r'ارابيكا ستار|اربيكا استار','ارابيكا ستار','ar_coffeeName')
    fix_words(r'دانكن','دانكن دونتس','ar_coffeeName')
    fix_words(r'كرسبي','كرسبي كريم','ar_coffeeName')
    fix_words('واينز','واينز','ar_coffeeName')
    fix_words(r'وقت الدونت دونت تايم','وقت الدونت','ar_coffeeName')
    fix_words(r'بوكس|بكس','ستاربكس','ar_coffeeName')
    fix_words(r'درافت','درافت','ar_coffeeName')
    fix_words(r'وكف','وكف','ar_coffeeName')
    fix_words('بارولي','بارولي','ar_coffeeName')

# function for fixing specific shop names 
def fix_words(contained_word, true_word,col):
    mask = cafes_text[col].str.contains(contained_word, case=False, na=False)
    cafes_text.loc[mask, col] = true_word

    if 'ar' in col:
        cleaned_words_ar.append(true_word)
    else:
        cleaned_words_eng.append(true_word)

# function for printing urls(google maps)
def print_urls_for_shop(cafes, shop_name,col):
    filtered = cafes[cafes[col] == shop_name]['url']
    for i, j in filtered.items():
        print(i, j)
        
# function to get the matching words based on string pattern
def print_matching_coffee_names(pattern,col):
    matches = cafes[cafes[col].str.contains(pattern, regex=True, na=False)].coffeeName.unique().tolist()
    for name in matches:
        print(name)

translations = {
    'kyan':'كيان',
    'starbucks':'ستاربكس',
    'shine':'بريق',
    'star sky': 'ستار سكاي',
    'veloce':'فيلوتشي',
    'joffreys':'جوفريز',
    'nivana':'نيفانا',
    'narees':'ناريز',
    'moon':'مون',
    'drip':'المقطره',
    'arabica':'ارابيكا',
    'window':'نافذه',
    'krispy kreme':'كرسبي كريم',
    'cava cod':'كافا كود',
    'waynes':'واينز',
    '&':'&',
    'jolt':'جولت',
    'liger':'لايقر',
    'passion':'باشن',
    'alfies':'الفيز',
    'half million':'هاف مليون',
    'no name':'بدون اسم',
    'mix':'مكس',
    'tutti':'توتي',
    'mccafe':'ماك',
    'barns':'بارنز',
    'dunkin donuts':'دانكن دونتس',
    'address':'عنوان',
    'dr.cafe':'د.كيف',
    'java':'جافا',
    'java time':'جافا تايم',
    'dan':'دان',
    'dana':'دانه',
    'mammabunz':'ماما بنز',
    'mezaj magribi':'مزاج مغربي',
    'taj':'تاج',
    'costa':'كوستا',
    'tim hortons':'تيم هورتنز',
    'sors': 'سورس',
    'arabica star':'ارابيكا ستار',
    'key':'كي',
    'draft':'درافت',
    'moroccan taste':'المذاق المغربي',
    'dose':'دوز',
    '4twins':'فورتوينز',
    'day':'يوم',
    'hi':'هاي',
    'shine':'بريق',
    'molten chocolate':'مولتن شوكلت',
    'cofen':'كوفن',
    'ounce':'اونصه',
    'iv':'اي في',
    'mantorose':'مانتوروز',
    'overdose':'اوفردوز',
    'moroccan':'المغرب',
    'cinnabon':'سينابون',
    'recipe':'ريسبي',
    'fast':'فاست',
    'brew crew':'برو كرو',
    'five elephants':'فايف اليفانتس',
    'luca':'لوكا',
    'sunset': 'صن ست',
    'veto':'فيتو',
    'urth':'ايرث',
    'style':'ستايل',
    'voltage':'فولتاج',
    'black line':'بلاك لاين',
    'next':'نكست',
    'mercato': 'ميركاتو',
    'onda':'اوندا',
    'crepe & waffle':'كريب وافل',
    'tale':'تيل',
    'bene':'بيني',
    'caffa':'كافا',
    'gloria jeans':'غلوريا جينز',
    'service': 'خدمة',
    'balcony':'بالكوني',
    'gourmet':'جورميه',
    'saheel':'صهيل',
    'm dee':'ام دي',
    'bon':'بن',
    'santorini':'سانتوريني',
    '8oz':'ايت اوز',
    'onza':'اونزا',
    'wacafe':'وكف',
    'parole':'بارولي',
    'pavla':'بافلا',
    'jaam':'جام',
    'arbista': 'أربيستا',
    'white mustache':'وايت موستاش',
    'knoll roasters':'محمصه نول',
    'coffeeholics roastery':'محمصة كوفي هوليكس'
}

### Cleaning the coffeeName column

In [4]:
cafes_text = cafes.copy()[['id','raw_coffeeName']]

#### Split by language & apply cleaning

In [5]:
cafes_text[['eng_coffeeName', 'ar_coffeeName']] = cafes_text['raw_coffeeName'].apply(split_by_language)
cafes_text['eng_coffeeName'] = cafes_text['eng_coffeeName'].apply(clean_str_eng)
cafes_text['ar_coffeeName'] = cafes_text['ar_coffeeName'].apply(clean_str_ar)

#### Remove Resorts, Tea Places, Warehouses, and fast food

In [6]:
mask_eng = ~cafes_text['eng_coffeeName'].str.contains(r'tea|tea4|resort|warehouse|tea|karak|fast\s?food|resthouse|parking', case=False, na=False)
mask_ar = ~cafes_text['ar_coffeeName'].str.contains(r'ريزورت|استراحه|شاي|شاهي|كرك|تي|تيه', case=False, na=False)
cafes_text = cafes_text[(mask_eng) & (mask_ar) ].copy()
cafes_text.tail()


,id,raw_coffeeName,eng_coffeeName,ar_coffeeName
2604,2605,Double one,double one,
2605,2606,Saws speciality coffee | ساوس للقهوة المختصة,saws,ساوس
2606,2607,عنوان القهوة,,عنوان
2607,2608,مقهى سوداني maqaa sudaniun,maqaa sudaniun,سوداني
2608,2609,A12 Cafe | اي تويلف,a12,اي تويلف


#### English Text Cleaning

In [7]:
specific_eng_names()

/var/folders/cl/4ndfg5191d74xd8pclttf16w0000gn/T/ipykernel_57346/770947766.py:177: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  mask = cafes_text[col].str.contains(contained_word, case=False, na=False)


#### Arabic Text Cleaning

In [8]:
specific_ar_names()

#### Translations

In [9]:
# for matches in english words, fill in its arabic values
for eng, ar in translations.items():
    mask = cafes_text['eng_coffeeName'] == eng
    cafes_text.loc[mask, 'ar_coffeeName'] = ar


# for matches in arabic words, fill in its english values
reverse_translations = {v: k for k, v in translations.items()}
for ar, eng in reverse_translations.items():
    mask = cafes_text['ar_coffeeName'] == ar
    cafes_text.loc[mask, 'eng_coffeeName'] = eng

In [10]:
empty_both = cafes_text[
    (cafes_text['eng_coffeeName'].str.strip() == '') &
    (cafes_text['ar_coffeeName'].str.strip() == '')
]
# what to do here?
empty_both

,id,raw_coffeeName,eng_coffeeName,ar_coffeeName
189,190,..,,
322,323,كوفي شوب,,
369,370,مقهى كيف,,
674,675,coffe shop,,
676,677,Coffe,,
819,820,كوفي,,
1091,1092,coffee shop,,
1109,1110,كوفي شوب,,
1368,1369,مقهى,,
1487,1488,coffee shop,,


Replace Empty Strings with np.nan

In [16]:
cols = ['eng_coffeeName', 'ar_coffeeName']
cafes_text[cols] = cafes_text[cols].replace('', np.nan)

In [19]:
cafes_text = cafes_text[['id','eng_coffeeName','ar_coffeeName']]
cafes_text.head()

,id,eng_coffeeName,ar_coffeeName
0,1,cacti,NaN
1,2,NaN,فازا
2,3,narees,ناريز
3,4,fc lounge,اف سي لاونج
4,5,peaks,NaN


#### Coordinates Data Cleaning

Removing some rows that have duplicate lat and long(not all tho)

In [20]:
cafes_rest= cafes.drop('raw_coffeeName',axis=1)

# dropping 4 rows with null latitude and longitude
cafes_rest = cafes_rest.dropna()

#### Joining back to get full cleaned dataframe

In [21]:
cleaned_cafes = cafes_text.merge(cafes_rest,on='id')
cleaned_cafes.head()

,id,eng_coffeeName,ar_coffeeName,rating,rating_count,url,24_hours,longitude,latitude
0,1,cacti,NaN,4.2,2212,https://www.google.com/maps/place/Cacti+Cafe/d...,True,46.735613,24.833986
1,2,NaN,فازا,4.3,889,https://www.google.com/maps/place/%D9%81%D8%A7...,False,46.697337,24.772458
2,3,narees,ناريز,3.8,39,https://www.google.com/maps/place/%D9%86%D8%A7...,False,46.852109,24.807277
3,4,fc lounge,اف سي لاونج,3.5,539,https://www.google.com/maps/place/Fc+Lounge+-+...,True,46.768059,24.813115
4,5,peaks,NaN,4.6,25,https://www.google.com/maps/place/PEAKS/data=!...,False,46.634809,24.742045
